# Rotating popcorn
This Notebook launches a condition number study for the rotating popcorn testcase as presented in the paper "Cell agglomeration strategy for eXtended discontinous Galerkin methods" (2024) 
 The calculations can be very long, in order to use a HPC cluster ``..\.BoSSS\etc\BatchProcessorConfig.json`` must be modified for the execution queue.

In [1]:
#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [2]:
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
buildname = String.IsNullOrEmpty(buildname)? "Popcorn2D" : buildname;
dbname = String.IsNullOrEmpty(dbname)? "condStudyBDF3" : dbname; 
string table_name = String.Concat(buildname, "_", dbname);

In [3]:
int SpaceDim = 2;
double _partRad = 0.6;

In [4]:
// set parameterz
int[] core_sweep = { 1};
int[] PolyDegS = new int[] {1, 2, 3};
int[] ResArray = new int[] {16, 32, 64, 128};
double[] aggThresolds = new double[] {0, 0.1, 0.2, 0.3, 0.4, 0.5};

In [5]:
int NoOfTimeSteps = 400;
bool Steady = false;
bool IncludeConvection = true;
var Gshape = Shape.Popcorn;

In [6]:
ExecutionQueues

index,type,value
0,BoSSS.Application.BoSSSpad.SlurmClient,"SlurmClient q0-Lichtenberg user storage : mt64qocy@lcluster18.hrz.tu-darmstadt.de, Slurm account: special00006"
1,BoSSS.Application.BoSSSpad.SlurmClient,"SlurmClient q1-Lichtenberg scratch : mt64qocy@lcluster14.hrz.tu-darmstadt.de, Slurm account: special00006"
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client q2-BoSSS-Convergence @DC3, @\\dc3\userspace\toprak\BoSSS-Convergence"
3,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client q3-BoSSS-ConvergenceFinal @DC3, @\\dc3\userspace\toprak\BoSSS-ConvergenceFinal"
4,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client q4-BoSSS-ConditionFinal @DC3, @\\dc3\userspace\toprak\BoSSS-ConditionFinal"
5,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client q5-BoSSS-ConvSteadyFinal @DC3, @\\dc3\userspace\toprak\BoSSS-ConvSteadyFinal"
6,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,MiniBatchProcessor client LocalPC @C:\localBatch


In [7]:
var myBatch = ExecutionQueues[4];

In [8]:
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

{ Session Count = 0; Grid Count = 0; Path = \\dc3\userspace\toprak\BoSSS-ConditionFinal\Popcorn2D_test }

using grid with

In [9]:
double xMax = 1.0, yMax = 1.0, zMax = 1.0;
double xMin = -1.0, yMin = -1.0,zMin = -1.0;

In [10]:
List<IGridInfo> Grids = new List<IGridInfo>();

foreach (var Res in ResArray) {
    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);    
    GridCommons grd;
    string gname = SpaceDim == 2 ? "RotPopcorn2D_1BCs" : "RotPopcorn3D_1BCs";
    
    var tmp = new List<IGridInfo>();
    foreach(var grid in myDB.Grids){
        try{
            if (String.IsNullOrEmpty(grid.Name))
                continue;

            Console.WriteLine("Found {0} with {1} cells on the database",grid.Name,grid.NumberOfCells);
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells == (_xNodes.Length - 1) * (_yNodes.Length - 1) * (SpaceDim == 3 ? GenericBlas.Linspace(zMin, zMax, Res + 1).Length - 1 : 1);
            if(IsMatch) 
            tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }

    Console.WriteLine("Found {0} grids for Res={1}", tmp.Count, Res);
    if (tmp.Any()) {
        Console.WriteLine("Chosing the first one found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.First());
    } else {
        if (SpaceDim == 3) {
            grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes);
        } else {
            grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes);
        }

        grd.Name = gname;
        grd.EdgeTagNames.Add(1, "Pressure_Outlet");
        grd.DefineEdgeTags(delegate (double[] X) {
            return 1;
        });
        myDB.SaveGrid(ref grd);
        Grids.Add(grd);
    }
}

Console.WriteLine("\nThe grids to be simulated:");
Grids


#0: { Guid = 7daf0905-c84f-4e57-9564-77cd1d1d9c50; Name = RotPopcorn2D_1BCs; Cell Count = 256; Dim = 2 }


## Generate Control object

### governing equations
- incompressible transient Navier-Stokes:
<br>$\frac{\partial \rho \vec{u}}{\partial t}+ \nabla \cdot ( \rho \vec{u} \otimes \vec{u}) + \nabla p - \eta \Delta \vec{u} = \vec{f} \ \ in \ \ \Omega_F(t) \times (0,T)$
<br>$\nabla \cdot \vec{u} = 0 \quad in \ \ \Omega_F(t) \times (0,T)$
- with inital conditions:
<br>$\vec{u}(\vec{x},0)  =  \vec{0} \quad on \ \ \Omega_F(0)$
<br>$p(\vec{x},0)  =  0 \quad  on \ \ \Omega_F(0)$
- with boundary conditions:
<br>$\vec{u}(\vec{x},t) = \vec{u}_{body} = \boldsymbol{\omega}(t) \times \vec{r} \quad on \ \ \mathcal{J} = \partial \Omega_S \cap \partial \Omega_F$
<br>$p \mathbf{I} - \frac{1}{Re} \nabla \vec{u} \vec{n}_{ \Gamma_{pOut} } = 0 \ \ on \ \ \Gamma_{pOut} = \partial \Omega \backslash \Gamma_{Inlet} $ 



- angular velocity of rotating popcorn $\boldsymbol{\omega}(t)=\frac{Re*\mu_A}{\rho_A*r_{max}*(2*r_p)}$

In [11]:
Func<IGridInfo, int, double, double, string, bool, bool, double, XNSE_Control> GenXNSECtrl = delegate(IGridInfo grd, int k, double Viscosity, double tiltAngle, string RotAxis, bool AMR, bool LoadBalance, double AggThrs){
    XNSE_Control C = new XNSE_Control();

    // basic database options
    // ======================
    C.SetDatabase(myDB);
    C.savetodb = true;
    int J  = grd.NumberOfCells;

    // Physical Parameters
    // ===================
    const double rhoA = 1;
    const double Re = 1000;
    double muA = Viscosity;
    double partRad = _partRad;
    double partDia = 2*partRad;
    double VelocityMax = Re * muA / rhoA / partDia;
    double anglev = VelocityMax / (partRad*1.39); // longest arm: partRad*1.39
    double period = 2*Math.PI/anglev;
    double ts = period/NoOfTimeSteps;
    double[] pos = new double[SpaceDim]; //position of the center is set to origin (0,0)
    string rotAxis= RotAxis ?? "z";

    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    // Meta data
    // ============    
    C.SessionName = string.Format("J{0}_k{1}_Re{2}_agg{3}_ti{4:f3}at{5}_r{6}_AMR{7}_LB{8}_W{9:f2}",
                             J, k,Re,AggThrs,tiltAngle,rotAxis.ToUpper(),_partRad,AMR,LoadBalance,anglev);

    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");

    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");

    // DG degrees and configuration
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 8));
    C.saveperiod = 10;
    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.Hilbert;
    C.DynamicLoadBalancing_On = LoadBalance;
    C.DynamicLoadBalancing_RedistributeAtStartup = false;
    C.DynamicLoadBalancing_Period = 10;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;
    C.ImmediatePlotPeriod = 5;
    C.SuperSampling = 0;

    // Set rigid body 
    // ============       
    double rateOfRadius = 0.0; //-1/(double)NoOfTimeSteps/ts;
    double ringRad = partRad/1.5;
    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim,ringRad,rateOfRadius: rateOfRadius, staticShape: Steady);
    C.SessionName += $"_dRdt{rateOfRadius:f2}";
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis(rotAxis);
    var tiltAxis = new Vector(1,0,0);
    C.Rigidbody.SetTilt(tiltAxis, tiltAngle);
    C.UseImmersedBoundary = true;
    
    // Set initial and boundary conditions
    // ============
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");

    // Solver parameters
    // ============
    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = AggThrs;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None; // idle level set (used for evolving LS)
    C.Timestepper_LevelSetHandling = LevelSetHandling.LieSplitting;
    C.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 0;
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.verbose = true;
    C.AdaptiveMeshRefinement = AMR;

    // Rotate only until symmetry axes, there is no need for further rotation
    double NoOfTimeStepsDouble = NoOfTimeSteps/5;

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = 1000;
        C.NoOfTimesteps = 1;
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = ts;        
        C.NoOfTimesteps = (int)NoOfTimeStepsDouble; 
    }
    
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3; 
    C.dtFixed = dt;
    C.SkipSolveAndEvaluateResidual = true; //to save computational time for condition study
    C.SessionName = "Solver" + !C.SkipSolveAndEvaluateResidual + "_" + C.SessionName;
    C.PlotAgglomeration = false;

    // //Condition number study
    // // ============
    var config = new OperatorAnalysisConfig();
    config.CalculateMassMatrix = true;
    config.CalculateGlobalConditionNumbers = true;
    config.CalculateStencilConditionNumbers = false;
    config.PlotStencilConditionNumbers = false;
    C.PostprocessingModules.Add(new BoSSS.Application.XNSE_Solver.Logging.CondLogger(config));
    C.SessionName += "_condStudy";

    return C;
};

In [12]:
//See an example control object
GenXNSECtrl(Grids[0],1,1,0.0,"z",false,false,0)

UseImmersedBoundary,True
TimesteppingMode,Transient
Timestepper_LevelSetHandling,LieSplitting
ControlFileText,<null>
GeneratedFromCode,False
ControlFileText_Index,0
NoOfMultigridLevels,1000000
FieldOptions,"[ [Velocity*, BoSSS.Solution.Control.FieldOpts], [Pressure, BoSSS.Solution.Control.FieldOpts], [PhiDG, BoSSS.Solution.Control.FieldOpts], [Phi, BoSSS.Solution.Control.FieldOpts], [Phi2DG, BoSSS.Solution.Control.FieldOpts], [Phi2, BoSSS.Solution.Control.FieldOpts], [Curvature, BoSSS.Solution.Control.FieldOpts] ]"
BoundaryValues,"[ [Pressure_Outlet, BoSSS.Solution.Control.AppControl+BoundaryValueCollection] ]"
BoundaryValueChanges,[ ]
InitialValues_EvaluatorsVec,"[ [Phi, BoSSS.Foundation.ScalarFunctionTimeDep], [Pressure, BoSSS.Foundation.ScalarFunctionTimeDep] ]"


# Submit jobs into cluster
Create the control objects for the sessions

In [13]:
var ViscositySweep = new double[]{1E-2};
var tiltAngles = new double[]{0}; //Math.PI/4
var axes = new string[]{"z"};
var AMRarray = new bool[] {false};
var LoadBalancearray = new bool[] {false};

List<XNSE_Control> controls = new List<XNSE_Control>();
foreach(IGridInfo grd in Grids){
    foreach(int k in PolyDegS){
        foreach(double v in ViscositySweep)
            foreach(double tiltAngle in tiltAngles)
                foreach(string axis in axes)
                    foreach(bool AMR in AMRarray)
                        foreach(bool LoadBalance in LoadBalancearray)
                            foreach(double agg in aggThresolds)
                               controls.Add(GenXNSECtrl(grd,k,v,tiltAngle,axis,AMR,LoadBalance,agg));
    }
}

List sessions to be submitted

In [14]:
controls.Select(s=>s.SessionName + "\n")

[ SolverFalse_J256_k1_Re1000_agg0.1_ti0.000atZ_r0.6_AMRFalse_LBFalse_W9.99_NSE_dRdt0.00_condStudy
 ]

Submit jobs

In [15]:
int k=1;
foreach(var ctrl in controls){
        string sessname = ctrl.SessionName;
        foreach(int cores in core_sweep){
            ctrl.SessionName = sessname + "_ct"+cores; 
                    var aJob   = new Job("Rot"+Gshape+"_"+ctrl.SessionName,typeof(XNSE));

            aJob.SetControlObject(ctrl);
            aJob.NumberOfMPIProcs         = cores;
            aJob.UseComputeNodesExclusive = true;
            //aJob.ExecutionTime            = "12:00:00";
            aJob.NumberOfThreads = 4;
            aJob.Activate(myBatch);
            Console.WriteLine("Submitted {0}th",k) ;
            k++;           
        }       
}
    


List sessions submitted

In [16]:
wmg.DefaultDatabase.Sessions